# 11. Auditoria: Análise de Conformidade

Neste módulo de Auditoria, focaremos em casos de uso reais. O primeiro é a **Verificação de Conformidade**. Dado um texto (ex: uma transação ou descrição de despesa) e uma política/regra, o LLM deve julgar se está conforme ou não.

**Cenário Real:** Analisar justificativas de despesas de viagem contra a política de viagens da empresa.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

!pip install -qU langchain langchain-openai langchain-community

In [ ]:
import os
try:
    from google.colab import userdata
except ImportError:
    userdata = None
import getpass

try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Digite sua OpenAI API Key: ")

## 1. Definindo as Regras (Critérios)

Vamos definir a política da empresa.

In [ ]:
politica_viagem = """
1. Despesas com alimentação não podem exceder R$ 100,00 por refeição.
2. O uso de táxi/Uber só é permitido se não houver transporte público disponível ou se for após as 22h.
3. Bebidas alcoólicas não são reembolsáveis em nenhuma hipótese.
4. Todas as despesas devem ter nota fiscal legível.
"""

## 2. Criando o Analisador de Conformidade

Usaremos um `ChatPromptTemplate` para instruir o modelo a agir como um auditor.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

prompt = ChatPromptTemplate.from_template(
    """Você é um Auditor Interno rigoroso.
    
    Analise a seguinte despesa com base na Política de Viagem fornecida.
    
    ### POLÍTICA DE VIAGEM ###
    {politica}
    
    ### DESPESA ###
    {despesa}
    
    Responda com:
    1. STATUS: [CONFORME / NÃO CONFORME]
    2. JUSTIFICATIVA: Breve explicação citando a regra violada, se houver.
    """
)

chain = prompt | llm | StrOutputParser()

## 3. Executando Testes

Vamos testar com alguns casos.

In [ ]:
casos = [
    "Jantar no restaurante 'O Bom Garfo', valor R$ 85,00. Inclui um suco de laranja. Nota fiscal anexada.",
    "Almoço de negócios com cliente. Total R$ 150,00. Nota fiscal inclusa.",
    "Corrida de Uber às 14h para o aeroporto. Não verifiquei ônibus. Valor R$ 40,00.",
    "Happy hour com equipe. Valor R$ 200,00 sendo R$ 100,00 em cervejas."
]

for i, caso in enumerate(casos):
    print(f"--- CASO {i+1} ---")
    print(f"Descrição: {caso}")
    res = chain.invoke({"politica": politica_viagem, "despesa": caso})
    print(res)
    print("\n")

## 4. Evolução (Batch)

Para auditar milhares de linhas, usaríamos `chain.batch(lista_de_inputs)` para processar em paralelo.

## Conclusão

Este padrão simples pode ser escalado para analisar contratos, e-mails ou logs de sistemas, filtrando o que realmente precisa de atenção humana.